# 🐾 Step 4-5: Build the CNN Model

In this notebook, we will:
1. Understand each layer of a CNN
2. Build our simple CNN from scratch
3. Compile the model (set optimizer, loss, metrics)

## 4.1 Setup (same as before)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

# Settings
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = 3  # cat, dog, badger

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.20.0


## 4.2 Understanding CNN Layers

Before we build, let's understand what each layer does:

| Layer | What it does | Example |
|-------|--------------|----------|
| **Conv2D** | Detects features (edges, textures, shapes) | "Is there a claw mark here?" |
| **ReLU** | Activation function, adds non-linearity | Negative → 0, Positive → keep |
| **MaxPooling2D** | Reduces size, keeps strongest signals | Takes the max value in a 2x2 area |
| **Flatten** | Converts 2D to 1D for dense layer | Unwraps the image into a list |
| **Dense** | Fully connected layer, makes decisions | "Based on all features, this is a cat" |
| **Dropout** | Randomly disables neurons during training | Prevents overfitting |

## 4.3 Build Our Simple CNN

Our architecture:
```rust
Input (224x224x3)
    ↓
Conv2D(32) → ReLU → MaxPool → 112x112x32
    ↓
Conv2D(64) → ReLU → MaxPool → 56x56x64
    ↓
Conv2D(128) → ReLU → MaxPool → 28x28x128
    ↓
Flatten → Dense(128) → Dropout(0.5) → Dense(3) → Softmax
    ↓
Output: [cat_prob, dog_prob, badger_prob]
```

In [2]:
def build_simple_cnn():
    """
    Build a simple CNN for footprint classification.
    
    Returns:
        model: A Keras Sequential model
    """
    model = models.Sequential([
        # Input layer - specify the image shape
        layers.Input(shape=(224, 224, 3)),
        
        # First Conv Block: 32 filters
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Second Conv Block: 64 filters
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Third Conv Block: 128 filters
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten and Dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Prevents overfitting
        
        # Output layer: 3 classes with softmax
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

# Create the model
model = build_simple_cnn()
print("Model created successfully!")

Model created successfully!


In [3]:
# View model summary - shows all layers and parameters
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

## 4.4 Understanding the Summary

- **Output Shape**: Size after each layer (shrinks with pooling)
- **Param #**: Number of learnable weights
- **Total params**: All weights the model will learn (~2-3 million is normal for this size)

## 4.5 Compile the Model

Compiling sets up HOW the model learns:

| Setting | What we use | Why |
|---------|-------------|-----|
| **Optimizer** | Adam | Adaptive learning rate, works well |
| **Loss** | categorical_crossentropy | For multi-class classification |
| **Metrics** | accuracy | Easy to understand |

In [4]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled!")
print(f"Optimizer: Adam")
print(f"Loss function: Categorical Crossentropy")
print(f"Metrics: Accuracy")

Model compiled!
Optimizer: Adam
Loss function: Categorical Crossentropy
Metrics: Accuracy


## 4.6 Visualize the Architecture

In [5]:
# Optional: visualize model (requires graphviz)
try:
    tf.keras.utils.plot_model(
        model, 
        to_file='model_architecture.png', 
        show_shapes=True,
        show_layer_names=True
    )
    from IPython.display import Image
    display(Image('model_architecture.png'))
except:
    print("Graphviz not installed - skipping visualization")
    print("The model summary above shows the same information!")

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


FileNotFoundError: No such file or directory: 'model_architecture.png'

FileNotFoundError: No such file or directory: 'model_architecture.png'

<IPython.core.display.Image object>

## 4.7 Save the Model Building Function

We'll save this to a Python file so we can reuse it:

In [6]:
# Save model configuration for later use
model_config = {
    'input_shape': (224, 224, 3),
    'num_classes': 3,
    'conv_filters': [32, 64, 128],
    'dense_units': 128,
    'dropout_rate': 0.5
}

print("Model configuration:")
for key, value in model_config.items():
    print(f"  {key}: {value}")

Model configuration:
  input_shape: (224, 224, 3)
  num_classes: 3
  conv_filters: [32, 64, 128]
  dense_units: 128
  dropout_rate: 0.5


## ✅ Steps 4-5 Complete!

**What we built:**
- A 3-block CNN with Conv2D → ReLU → MaxPool
- Flatten + Dense layers for classification  
- Dropout to prevent overfitting
- Softmax output for 3 classes

**What each part does (for your presentation):**
- Conv2D: "Extracts features like edges and shapes"
- MaxPooling: "Reduces size while keeping important information"
- Dropout: "Prevents the model from memorizing the training data"
- Softmax: "Converts outputs to probabilities that sum to 1"

**Next step:** Train the model!